In [ ]:
!pip install langchain langchain-community langchain-google-genai langgraph

In [2]:
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [20]:
import os
import sys
import asyncio
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from IPython.display import display, Markdown

# --- Configuration ---
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", temperature=0.7)

# --- DEBUG ---
def inspect(title):
  def output(value):
    print(f"\n--- inspection of {title} ---")
    display(Markdown(value))
    return value
  return output

# --- Define Chain Components ---

# 1. Initial Generation: Creates the first draft of the product description.
# The input to this chain will be a dictionary, so we update the prompt template.
generation_chain = (
    ChatPromptTemplate.from_messages([
        ("system", "Write a short, simple product description for a new smart coffee mug."),
        ("user", "{product_details}")
    ])
    | llm
    | StrOutputParser()
    | inspect("Generated Description")
)

# 2. Critique: Evaluates the generated description and provides feedback.
critique_chain = (
    ChatPromptTemplate.from_messages([
        ("system", """Critique the following product description based on clarity, conciseness, and appeal.
        Provide specific suggestions for improvement."""),
        # This will receive 'initial_description' from the previous step.
        ("user", "Product Description to Critique:\n{initial_description}")
    ])
    | llm
    | StrOutputParser()
    | inspect("Critique Output")
)

# 3. Refinement: Rewrites the description based on the original details and the critique.
refinement_chain = (
    ChatPromptTemplate.from_messages([
        ("system", """Based on the original product details and the following critique,
        rewrite the product description to be more effective.

        Original Product Details: {product_details}
        Critique: {critique}

        Refined Product Description:"""),
        ("user", ".")
    ])
    | llm
    | StrOutputParser()
)


# --- Build the Full Reflection Chain (Refactored) ---
# This chain is structured to be more readable and linear.
full_reflection_chain = (
    RunnablePassthrough.assign(
        initial_description=generation_chain
    )
    | RunnablePassthrough.assign(
        critique=critique_chain
    )
    | refinement_chain
)


# --- Run the Chain ---
async def run_reflection_example(product_details: str):
    """Runs the LangChain reflection example with product details."""
    print(f"\n--- Running Reflection Example for Product: '{product_details}' ---")
    try:
        # The chain now expects a dictionary as input from the start.
        final_refined_description = await full_reflection_chain.ainvoke(
            {"product_details": product_details}
        )
        print("\n--- Final Refined Product Description ---")
        display(Markdown(final_refined_description))
    except Exception as e:
        print(f"\nAn error occurred during chain execution: {e}")

if __name__ == "__main__":
    test_product_details = "A mug that keeps coffee hot and can be controlled by a smartphone app."
    await run_reflection_example(test_product_details)


--- Running Reflection Example for Product: 'A mug that keeps coffee hot and can be controlled by a smartphone app.' ---

--- inspection of Generated Description ---


**Never sip lukewarm coffee again!** This smart mug keeps your drink at the perfect temperature, all controlled from your phone. Enjoy hot coffee, tea, or cocoa, anytime, anywhere.


--- inspection of Critique Output ---


Here's a critique of the product description, along with suggestions for improvement:

**Critique:**

*   **Clarity:** The description is fairly clear. It immediately conveys the core benefit (hot coffee) and mentions the key feature (temperature control via phone).
*   **Conciseness:** It's brief, which is good. However, it could be slightly more specific and impactful.
*   **Appeal:** It hints at an attractive benefit (perfectly hot drink), but it's a bit generic. It lacks a strong emotional hook or unique selling proposition. "Anytime, anywhere" is a common phrase and doesn't particularly excite.

**Suggestions for Improvement:**

Here's a revised description, incorporating the suggestions below:

**Option 1 (Focus on Problem/Solution):**

"**Tired of cold coffee?** This smart mug lets you savor every sip at your ideal temperature, controlled effortlessly from your smartphone. Enjoy perfectly hot coffee, tea, or cocoa all day long, whether you're at your desk or on the go."

**Changes & Rationale:**

*   **Stronger Opening:** Starts with a common pain point ("Tired of cold coffee?"). This immediately grabs the reader's attention.
*   **More Engaging Language:** Replaced "keeps your drink at the perfect temperature" with "lets you savor every sip at your ideal temperature." This emphasizes the enjoyment.
*   **Reinforced Benefit:** "Perfectly hot...all day long" adds emphasis and highlights a key advantage.
*   **Conciseness Maintained:** The core message is still delivered quickly.

**Option 2 (Focus on Features & Benefits):**

"**The Smart Mug That Knows Your Perfect Temperature.** Set your ideal drinking temperature directly from your phone and enjoy coffee, tea, or cocoa that stays perfectly hot for hours. Never settle for lukewarm again. Perfect for work, travel, or relaxing at home."

**Changes & Rationale:**

*   **Compelling Headline:** "The Smart Mug That Knows Your Perfect Temperature" is more intriguing.
*   **Clear Feature Highlight:** Directly mentions "Set your ideal drinking temperature directly from your phone."
*   **Benefit-Driven Language:** Emphasizes that the drink stays hot "for hours."
*   **Stronger Call to Action (Implied):** "Never settle for lukewarm again." is a more compelling and concise version.
*   **Location Suggestions:** provides additional benefits and application.

**Key improvements made in the revisions:**

*   **Stronger Opening Hooks:** Capture attention immediately by addressing a common problem or hinting at a unique selling point.
*   **More Vivid Language:** Use words that evoke a positive emotional response (e.g., "savor," "effortlessly").
*   **Focus on the Benefit:** Emphasize *what* the customer gains (e.g., enjoyment of the perfect temperature) rather than just the feature itself.
*   **Specific and Concise:** Provide more context.
*   **Stronger Call to Action:** Use language that implies the customer should try this product.

By implementing these suggestions, you can create a more compelling and effective product description that will resonate with potential customers.


--- Final Refined Product Description ---


Okay, based on the provided critique and suggestions, here are a few refined product descriptions, keeping in mind different potential target audiences:

**Option 1 (Concise & Benefit-Focused):**

"**Never drink cold coffee again.** This smart mug, controlled by your smartphone, lets you set and maintain your perfect drinking temperature. Enjoy hot coffee, tea, or cocoa, anytime, anywhere. Experience the ultimate in beverage enjoyment."

**Why this works:**

*   **Direct & Urgent:** Opens with a common problem and a strong call to action (never drink cold coffee again).
*   **Clear Benefit:** Focuses on the enjoyment aspect.
*   **Simple & Effective:** Delivers the core message concisely.
*   **Broad Appeal:** Suitable for a general audience.

**Option 2 (Feature-Rich & Tech-Savvy):**

"**Control Your Coffee, Your Way.** The smart mug that puts temperature control at your fingertips. Download the app, set your ideal temperature, and enjoy hours of perfectly hot coffee, tea, or cocoa. Features include customizable temperature profiles, auto-shutoff, and long-lasting battery life. Elevate your daily ritual."

**Why this works:**

*   **Intriguing Headline:** "Control Your Coffee, Your Way" implies personalization and convenience.
*   **Feature Highlight:** Clearly mentions the app control and key features.
*   **Tech-Focused Language:** Appeals to those interested in smart home devices.
*   **Benefit-Oriented:** "Elevate your daily ritual" adds a touch of aspirational marketing.

**Option 3 (Emphasis on Lifestyle & Convenience):**

"**Your Perfect Drink, Perfected.** Say goodbye to lukewarm. This smart mug, seamlessly controlled by your phone, keeps your favorite beverages at the ideal temperature, wherever life takes you. Perfect for busy mornings, long commutes, and relaxing evenings. Enjoy your coffee, tea, or cocoa at your perfect temperature and make the moment yours."

**Why this works:**

*   **Emphasis on Lifestyle:** Paints a picture of convenience and ease of use in different situations.
*   **Addresses a Common Problem:** Avoids lukewarm beverages.
*   **Benefit-Driven:** Focuses on the enjoyment and convenience aspects, framing the mug as a lifestyle accessory.
*   **Relatable Appeal:** Conveys the ultimate feeling of comfort.

**Key improvements across all options:**

*   **Stronger Hooks:** Starting with a pain point or a benefit.
*   **Concise and Engaging Language:** Using active verbs and more evocative words.
*   **Benefit-Focused:** Emphasizing what the customer *gets* rather than just the features.
*   **Clear Call to Action (Implied):** Encouraging the reader to consider the product.
*   **Consideration of Target Audience:** Tailoring the language to different potential customers.

The best option depends on the overall marketing strategy and target audience for this smart mug. Each option provides a compelling way to describe the product.